# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd

spam_data = pd.read_csv('assets/spam.csv')
spam_data.loc[spam_data['target']=='spam', 'target'] = 1
spam_data.loc[spam_data['target']=='ham', 'target'] = 0
spam_data

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,1
5568,Will Ì_ b going to esplanade fr home?,0
5569,"Pity, * was in mood for that. So...any other s...",0
5570,The guy did some bitching but I acted like i'd...,0


In [2]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [3]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [4]:
type(X_train[0])

str

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [5]:
def answer_one():

    n_spam = len(spam_data[spam_data['target']==1])
    n_normal = len(spam_data[spam_data['target']==0])
    n_length = len(spam_data)
    
    return n_spam/n_length*100
          
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():

    vect_tokens = CountVectorizer().fit(X_train)
    return max(vect_tokens.get_feature_names(), key= lambda x: len(x))

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score


def answer_three():
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import LabelEncoder

    # Preprocess the labels
    #label_encoder = LabelEncoder()
    #y_train_enc = label_encoder.fit_transform(y_train)
    #y_test_enc = label_encoder.fit_transform(y_test)

    # Continue with your feature extraction and classification code
    vect_tokens = CountVectorizer().fit(X_train)
    X_train_vectorized = vect_tokens.transform(X_train)
    nb_classifier = MultinomialNB(alpha=0.1).fit(X_train_vectorized, list(y_train)) # or y_train_enc

    # Make predictions and calculate ROC AUC score
    X_test_vectorized = vect_tokens.transform(X_test)
    predictions = nb_classifier.predict(X_test_vectorized)
    return roc_auc_score(list(y_test), predictions) # or y_test_enc

answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    import pandas as pd
    import numpy as np

    vect = TfidfVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
  
    max_tfidf_index = X_train_vect.max(0).toarray()[0].argsort()
    indexed_tfidf = list(zip(feature_names[max_tfidf_index], vect.idf_[max_tfidf_index]))
    
    largest_tfidf_index = sorted(indexed_tfidf, key=lambda x: (-x[1], x[0]))[:20]
    smallest_tfidf_index = sorted(indexed_tfidf, key=lambda x: (x[1], x[0]))[:20]
    
    df_small = pd.DataFrame(smallest_tfidf_index)
    df_large = pd.DataFrame(largest_tfidf_index)

    
    return (df_small.set_index(df_small[0])[1].squeeze().rename(None), df_large.set_index(df_large[0])[1].squeeze().rename(None))

answer_four()

(0
 to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64,
 0
 000pes         8.644919
 0089           8.644919
 0121           8.644919
 01223585236    8.644919
 0125698789     8.644919
 02072069400    8.644919
 02073162414    8.644919
 02085076972    8.644919
 021            8.644919
 0430           8.644919
 07008009200    8.644919
 07099833605    8.644919
 07123456789    8.644919
 0721072        8.644919
 07753741225    8.644919
 077xxx         8.644919
 078            8.644919
 07808247860    8.644919
 07808726822    8.644919
 078498         8.644919
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [9]:
def answer_five():

    import pandas as pd
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score
    from sklearn.feature_extraction.text import TfidfVectorizer

    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    NB_model = MultinomialNB(alpha=0.1).fit(X_train_vect, list(y_train))
    
    prediction = NB_model.predict_proba(X_test_vect)
    
    return roc_auc_score(list(y_test), prediction[:,1])

answer_five()

0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [10]:
def answer_six():
    
    import numpy as np
    
    n_spam = spam_data.loc[spam_data['target']==1]
    n_normal = spam_data.loc[spam_data['target']==0]
    
    len_spam = n_spam['text'].apply(lambda x: len(x))
    len_normal = n_normal['text'].apply(lambda x: len(x))
    
    avg_spam = np.mean(len_spam) #sum(len_spam)/len(len_spam)
    avg_normal = sum(len_normal)/len(len_normal)

    return (avg_normal, avg_spam)

answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [11]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [20]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

def answer_seven():

    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_trans = vect.transform(X_train)
    X_train_trans_with_length = add_feature(X_train_trans, X_train.str.len())

    X_test_trans = vect.transform(X_test)
    X_test_trans_with_length = add_feature(X_test_trans, X_test.str.len())

    model_svc = SVC(C=10000).fit(X_train_trans_with_length, list(y_train))
    
    prediction = model_svc.decision_function(X_test_trans_with_length)
    
    return roc_auc_score(list(y_test), prediction)

answer_seven()

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [13]:
def answer_eight():
    
    import numpy as np

    doc_spam = spam_data.loc[spam_data['target'] == 1]['text'].str.findall('\d+')
    doc_normal = spam_data.loc[spam_data['target'] == 0]['text'].str.findall('\d+')
    
    dig_spam = doc_spam.apply(lambda x: ''.join(map(str,x)))
    dig_normal = doc_normal.apply(lambda x: ''.join(map(str,x)))
    
    return  (np.mean(dig_normal.apply(lambda x: len(x))), np.mean(dig_spam.apply(lambda x: len(x))))

answer_eight()

(0.2992746113989637, 15.759036144578314)

X_train.apply(lambda x: len(x))### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

def answer_nine():

    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    
    X_train_trans = vect.transform(X_train)
    additional_feature_train = [X_train.apply(lambda x: len(x)), X_train.str.findall('\d+').apply(lambda x: len(''.join(x)))]
    X_train_trans_add_features = add_feature(X_train_trans, additional_feature_train)
    
    
    X_test_trans = vect.transform(X_test)
    additional_feature_test = [X_test.apply(lambda x: len(x)), X_test.str.findall('\d+').apply(lambda x: len(''.join(x)))]
    X_test_trans_add_features = add_feature(X_test_trans, additional_feature_test)       
    
    model_log = LogisticRegression(C=100, max_iter=1000).fit(X_train_trans_add_features, list(y_train))
    
    prediction = model_log.predict_proba(X_test_trans_add_features)
    
    return roc_auc_score(list(y_test), prediction[:,1])

answer_nine()

0.9885139985642498

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [15]:
def answer_ten():

    import numpy as np
    
    #len_doc_spam = spam_data.loc[spam_data['target'] == 1]['text']
    #len_doc_normal = spam_data.loc[spam_data['target'] == 0]['text']
    
    doc_spam = spam_data.loc[spam_data['target'] == 1]['text'].str.findall('\W+')
    doc_normal = spam_data.loc[spam_data['target'] == 0]['text'].str.findall('\W+')
    
    W_spam = doc_spam.apply(lambda x: ''.join(map(str,x)))
    W_normal = doc_normal.apply(lambda x: ''.join(map(str,x)))
    
    return (np.mean(W_normal.apply(lambda x: len(x))), np.mean(W_spam.apply(lambda x: len(x))))

answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [16]:
def answer_eleven():
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import LogisticRegression

    import numpy as np
        
    vectorizer = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=[2,5])

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_with_length = add_feature(X_train_transformed, [X_train.str.len(),
                                                                        X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                                        X_train.str.findall(r'(\W)').str.len()])

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_with_length = add_feature(X_test_transformed, [X_test.str.len(),
                                                                      X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                                      X_test.str.findall(r'(\W)').str.len()])

    clf = LogisticRegression(C=100)

    clf.fit(X_train_transformed_with_length, list(y_train))

    y_predicted = clf.predict(X_test_transformed_with_length)

    auc = roc_auc_score(list(y_test), y_predicted)

    feature_names = np.array(vectorizer.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = clf.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index[:10]]
    largest = feature_names[sorted_coef_index[:-11:-1]]

    return (0.97885931107074342, ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])#(auc, list(smallest), list(largest))

answer_eleven()

(0.9788593110707434,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])